### Free space

In this notebook we automatise the computation of matrix elements for later use by a Krylov iterative solver. We save all symbolic results in a file containing reusable functions: 
        
        freeSpaceME.py
        
This file only has to be generated once (per Green's function).

On the note of a general Green's function, have to write the matrix elements more general. In previous computation I have used a lot of the symmetries of the free space Green's function.

In [1]:
from sympy import *
import numpy as np
from sympy.printing.numpy import NumPyPrinter

import re

### Definitions and auxiliary functions

In [2]:
xij, yij, zij, b, eta = symbols("xij yij zij b eta", real=True) #probably need FH and VH symbols too

# indices for summation:
## Caution, cannot use lambda, instead use Lambda
## do not use eta as index, it's the viscosity
alpha, beta, gamma, gamma1, gamma2, gamma3, gamma4, kappa, kappa1, kappa2, mu, nu, nu1, nu2, Lambda, eta2, zeta, sigma = symbols(
    'alpha, beta, gamma, gamma1, gamma2, gamma3, gamma4, kappa, kappa1, kappa2, mu, nu, nu1, nu2, Lambda, eta2, zeta, sigma', integer=True)


## for FH and VH checkout the 'Indexed' package to define them as FH[1], FH[2] etc.

rij = Array([xij, yij, zij])

def norm(rij):
    return sqrt(summation(rij[alpha]**2,(alpha,0,2)))

KroneckerDelta = eye(3)

def Laplacian(G):
    return diff(G, rij[0], rij[0]) + diff(G, rij[1], rij[1]) + diff(G, rij[2], rij[2])

def eps(alpha, beta, gamma):
    return LeviCivita(alpha,beta,gamma)

In [3]:
def Curl(G, alpha, beta): ## the function to take the curl of and the two remaining indices 
    result=0
    for i in range(3):
        for j in range(3):
            result += eps(alpha,i,j)*diff(G(rij,j,beta),rij[i]) ## G is symmetric in its indices
    return result

def CurlCurl(G, alpha, beta): ## the function to take the curl of and the two remaining indices 
    result=0
    for i in range(3):
        for j in range(3):
            for k in range(3):
                for l in range(3):
                    result += eps(alpha,i,j)*eps(beta,k,l)*diff(G(rij,j,l), rij[i], rij[k])
    return result

### Matrix elements

In [4]:
def G(rij, alpha, beta):
    return 1/(8*pi*eta)*(KroneckerDelta[alpha,beta]/norm(rij) + rij[alpha]*rij[beta]/norm(rij)**3)

def delG(rij, alpha, beta, gamma):
    return 1/(8*pi*eta)*((-KroneckerDelta[alpha,beta]*rij[gamma] + KroneckerDelta[alpha,gamma]*rij[beta] + KroneckerDelta[beta,gamma]*rij[alpha])/norm(rij)**3
                         - 3*rij[alpha]*rij[beta]*rij[gamma]/norm(rij)**5)

def lapG(rij, alpha, beta):
    return 1/(4*pi*eta)*(KroneckerDelta[alpha,beta]/norm(rij)**3 - 3*rij[alpha]*rij[beta]/norm(rij)**5)

In [5]:
##auxiliary functions
def dellapG(rij, alpha, beta, kappa1):
    return diff(lapG(rij, alpha, beta), rij[kappa1])

def deldelG(rij, zeta, beta, kappa1, nu):
    return diff(delG(rij, zeta, beta, kappa1), rij[nu])

def deldellapG(rij, alpha, beta, gamma1, kappa1):
    return diff(dellapG(rij,alpha,beta, gamma1), rij[kappa1])

def deldeldelG(rij, alpha, beta, gamma1, gamma2, gamma3):
    return diff(deldelG(rij, alpha, beta, gamma1, gamma2), rij[gamma3])

def deldeldeldelG(rij, alpha, beta, gamma1, gamma2, gamma3, gamma4):
    return diff(deldeldelG(rij, alpha, beta, gamma1, gamma2, gamma3), rij[gamma4])

In [6]:
# GLL
def G1s1s(rij, alpha, beta):
    return G(rij, alpha, beta) + b**2/3*lapG(rij, alpha, beta)

def G1s2a(rij, alpha, beta):
    return -0.5*b*Curl(G, beta, alpha)
#summation(summation(eps(beta, nu, zeta)*delG(rij, alpha, zeta, nu), (nu,0,2)), (zeta,0,2))

def G2a1s(rij, alpha, beta):
    return b*Curl(G,alpha,beta)
#summation(summation(eps(alpha,nu,zeta)*delG(rij,zeta,beta,nu), (nu,0,2)), (zeta,0,2))

def G2a2a(rij, alpha, beta):
    return -0.5*b*b*CurlCurl(G,alpha,beta)
#summation(summation(summation(summation(eps(alpha,mu,kappa)*eps(beta,nu,zeta)*deldelG(rij,kappa,zeta,nu,mu),(mu,0,2)), (kappa,0,2)), (nu,0,2)), (zeta,0,2))

In [7]:
## GL2s and G2sL
def G1s2s(rij, alpha, kappa1, beta):
    return -0.5*b*((delG(rij, alpha, beta, kappa1) + delG(rij, alpha, kappa1, beta)) + 4*b**2/15*(dellapG(rij, alpha, beta, kappa1) + dellapG(rij, alpha, kappa1, beta)))

def G2a2s(rij, alpha, kappa1, beta):
    return -0.5*b*b*(diff(Curl(G, alpha, beta),rij[kappa1]) + diff(Curl(G, alpha, kappa1),rij[beta]))
#summation(summation(eps(alpha,nu,zeta)*(deldelG(rij, zeta, beta, kappa1, nu) + deldelG(rij, zeta, kappa1, beta, nu)), (nu,0,2)), (zeta,0,2))

def G2s1s(rij, alpha, gamma1, beta):
    return 0.5*b*((delG(rij, alpha, beta, gamma1) + delG(rij, gamma1, beta, alpha)) + 4*b**2/15*(dellapG(rij,alpha,beta,gamma1) + dellapG(rij, gamma1, beta, alpha)))

def G2s2a(rij, alpha, gamma1, mu):
    return -0.25*b*b*(diff(Curl(G,mu,gamma1), rij[alpha]) + diff(Curl(G,mu,alpha), rij[gamma1])) #sign change from -> eps(mu, kappa1, beta)
#summation(summation(eps(beta, kappa1, mu)*(deldelG(rij, gamma1, beta, alpha, kappa1) + deldelG(rij, alpha, beta, gamma1, kappa1)), (kappa1, 0, 2)), (beta,0,2))


## G2s2s
def G2s2s(rij, alpha, gamma1, kappa1, beta):
    return -0.25*b*b*((deldelG(rij, alpha, beta, gamma1, kappa1) + deldelG(rij, gamma1, beta, alpha, kappa1)) + 
                      (deldelG(rij,alpha,kappa1, gamma1, beta) + deldelG(rij, gamma1, kappa1, alpha, beta)) +
                      b**2/5*(deldellapG(rij,alpha,beta,gamma1,kappa1) + deldellapG(rij,gamma1,beta,alpha,kappa1)) + 
                      b**2/5*(deldellapG(rij, alpha, kappa1, gamma1, beta) + deldellapG(rij,gamma1,kappa1,alpha,beta)))


## higher matrix elements
def G1s3t(rij, alpha, beta):
    return -1/5*b**2*lapG(rij,alpha,beta)


# symmetry for matrix elements of form l sigma, 3s
def g13sSym(rij,alpha,kappa2,kappa1,beta):
    return 1/3*(deldelG(rij,alpha,beta,kappa1,kappa2) + deldelG(rij,alpha,kappa2,kappa1,beta) + deldelG(rij,alpha,kappa1,beta,kappa2)) - 1/15*(
        KroneckerDelta[kappa1,kappa2]*lapG(rij,alpha,beta) + KroneckerDelta[kappa1,beta]*lapG(rij,alpha,kappa2) + KroneckerDelta[beta,kappa2]*lapG(rij,alpha,kappa1))

def G1s3s(rij,alpha,kappa2, kappa1, beta):
    return b**2*(g13sSym(rij,alpha,kappa2,kappa1,beta) + 5*b**2/21*Laplacian(g13sSym(rij,alpha,kappa2,kappa1,beta)))


# symmetry of matrix elements of form 3s, l sigma
def g3s1Sym(rij,alpha,gamma1,gamma2,beta):
    return 1/3*(deldelG(rij,alpha,beta,gamma1,gamma2) + deldelG(rij,gamma1,beta,gamma2,alpha) + deldelG(rij,gamma2,beta,alpha,gamma1)) - 1/15*(
        KroneckerDelta[gamma1, gamma2]*lapG(rij,alpha,beta) + KroneckerDelta[gamma2,alpha]*lapG(rij,gamma1,beta) + KroneckerDelta[alpha,gamma1]*lapG(rij,gamma2,beta))

def G3s1s(rij,alpha,gamma1,gamma2,beta):
    return b**2*(g3s1Sym(rij,alpha,gamma1,gamma2,beta) + 5*b**2/21*Laplacian(g3s1Sym(rij,alpha,gamma1,gamma2,beta)))

def G3a1s(rij,Lambda, kappa2, beta):
    return 0.5*b**2*(diff(Curl(G, Lambda, beta), rij[kappa2]) + diff(Curl(G,kappa2,beta), rij[Lambda]))
#summation(summation(eps(Lambda, gamma1, alpha)*deldelG(rij,alpha,beta,gamma1,kappa2) +
#                    eps(kappa2, gamma1, alpha)*deldelG(rij,alpha,beta,gamma1,Lambda), (gamma1,0,2)), (alpha,0,2))

def G1s3a(rij, alpha, nu2, sigma):
    return 2/3*b**2*(diff(Curl(G,sigma, alpha),rij[nu2]) - 1/3*summation(eps(sigma,nu2,beta)*lapG(rij,alpha,beta), (beta,0,2))) # sign change due to --> eps(sigma,nu1,beta)
#summation(summation(eps(beta,nu1,sigma)*(deldelG(rij,alpha,beta,nu1,nu2) - 1/3*KroneckerDelta[nu1,nu2]*lapG(rij, alpha,beta)), (beta,0,2)), (nu1,0,2))

def g3sCurlSym(rij,alpha,gamma1,gamma2,mu):
    return 1/3*(diff(Curl(G,mu,alpha), rij[gamma1],rij[gamma2]) + diff(Curl(G,mu,gamma1), rij[gamma2],rij[alpha]) + diff(Curl(G,mu,gamma2), rij[alpha],rij[gamma1])) 
#- 1/15*(KroneckerDelta[gamma1, gamma2]*Laplacian(Curl(G,mu,alpha)) +KroneckerDelta[gamma2, alpha]*Laplacian(Curl(G,mu,gamma1)) +
#      KroneckerDelta[alpha, gamma1]*Laplacian(Curl(G,mu,gamma2)))    ##this part is zero anyway
        

def G3s2a(rij, alpha,gamma1,gamma2,mu):
    return -0.5*b**3*g3sCurlSym(rij, alpha, gamma1, gamma2, mu)## sign change due to -> eps(mu, kappa1, beta)
#0.5*b^3*Sum[Sum[eps[beta, kappa1, mu]*D[g3s1Sym[rij, alpha, gamma1, gamma2, beta], rij[kappa1]], {kappa1, 3}], {beta, 3}]

def G3a2a(rij, Lambda, kappa2, mu):
    return -0.25*b**3*(diff(CurlCurl(G,mu,Lambda), rij[kappa2]) + diff(CurlCurl(G,mu,kappa2), rij[Lambda])) #sign change -> eps(mu,kappa1,beta)


def k2s2sSym(rij,alpha,gamma1,kappa1,beta):
    return deldelG(rij,alpha,kappa1,gamma1,beta) + deldelG(rij,alpha,beta,gamma1,kappa1) + deldelG(rij,gamma1,kappa1,alpha,beta) + deldelG(rij,gamma1,beta,alpha,kappa1)

def K2s2s(rij,alpha,gamma1,kappa1,beta):
    return -2*pi*eta*b**3/3*(k2s2sSym(rij,alpha,gamma1,kappa1,beta) + b**2/5*Laplacian(k2s2sSym(rij,alpha,gamma1,kappa1,beta)))


def k3s2sSym(rij,alpha,gamma1,gamma2,beta,gamma):
    return diff(g3s1Sym(rij,alpha,gamma1,gamma2,gamma), rij[beta]) + diff(g3s1Sym(rij,alpha,gamma1,gamma2,beta), rij[gamma])

def K3s2s(rij,alpha,gamma1,gamma2,beta,gamma):
    return -4*pi*eta*b**4/3*(k3s2sSym(rij,alpha,gamma1,gamma2,beta,gamma) + 6*b**2/35*Laplacian(k3s2sSym(rij,alpha,gamma1,gamma2,beta,gamma)))


def K3a2s(rij, Lambda, kappa2, beta, gamma):
    return -2*pi*eta*b**4/3*( (diff(Curl(G,Lambda,gamma), rij[beta], rij[kappa2]) + diff(Curl(G,Lambda,beta), rij[gamma], rij[kappa2])) + 
                               (diff(Curl(G,kappa2,gamma), rij[beta], rij[Lambda]) + diff(Curl(G,kappa2,beta), rij[gamma], rij[Lambda])))


def k1s2sSym(rij, alpha, beta, gamma):
    return delG(rij,alpha,gamma,beta) + delG(rij, alpha, beta, gamma)

def K1s2s(rij, alpha,beta,gamma):
    return -4*pi*eta*b**2/3*(k1s2sSym(rij, alpha, beta, gamma) + Laplacian(k1s2sSym(rij, alpha, beta, gamma)))

def K1s3t(rij, alpha,beta):
    return -6*pi*eta*b**3/25*lapG(rij, alpha, beta)

def K2s3t(rij,alpha, gamma1, nu2):
    return -3*pi*eta*b**4/25*(dellapG(rij,alpha,nu2,gamma1) + dellapG(rij, gamma1, nu2, alpha))

def K3s3t(rij, alpha, gamma1, gamma2, nu2):
    return -6*pi*eta*b**5/25*Laplacian(g3s1Sym(rij,alpha,gamma1,gamma2,nu2))


def G2s3t(rij,alpha,gamma1,nu2):
    return -b**3/10*(dellapG(rij,alpha,nu2,gamma1) + dellapG(rij,gamma1,nu2,alpha))

def G3s3t(rij,alpha,gamma1,gamma2,nu2):
    return -b**4/5*Laplacian(g3s1Sym(rij,alpha,gamma1,gamma2,nu2))


def G2s3a(rij,alpha,gamma1,mu,nu2): #sign change -> eps(mu,nu1,beta)
    return b**3/3*( diff(Curl(G, mu,alpha), rij[gamma1], rij[nu2]) - 1/3*diff(summation(eps(mu,nu2,beta)*Laplacian(G(rij,alpha,beta)), (beta,0,2)), rij[gamma1])
                   + diff(Curl(G, mu,gamma1), rij[alpha], rij[nu2]) - 1/3*diff(summation(eps(mu,nu2,beta)*Laplacian(G(rij,gamma1,beta)), (beta,0,2)), rij[alpha]))

def G3a3a(rij, Lambda, kappa2, mu, nu2): #sign change -> eps(mu,nu1,beta)
    return b**4/3*(diff(CurlCurl(G,mu,Lambda), rij[nu2], rij[kappa2]) + diff(CurlCurl(G,mu,kappa2), rij[nu2], rij[Lambda]))


def G3s3a(rij, alpha,gamma1,gamma2,mu,nu2): #sign change -> eps(mu,nu1,beta)
    return 2*b**4/3*(diff(g3sCurlSym(rij,alpha,gamma1,gamma2,mu), rij[nu2]) - 1/3*Laplacian(summation(eps(mu,nu2,beta)*g3s1Sym(rij,alpha,gamma1,gamma2,beta), (beta,0,2))))


def g2s3sSym(rij,alpha,gamma1,kappa2, kappa1, beta):
    return diff(g13sSym(rij,alpha,kappa2,kappa1,beta), rij[gamma1]) + diff(g13sSym(rij, gamma1, kappa2, kappa1, beta), rij[alpha])

def G2s3s(rij,alpha,gamma1,kappa2, kappa1, beta):
    return 0.5*b**3*(g2s3sSym(rij,alpha,gamma1,kappa2, kappa1, beta) + 6*b**2/35*Laplacian(g2s3sSym(rij,alpha,gamma1,kappa2, kappa1, beta)))


def G3a3s(rij,Lambda, eta2, kappa2, kappa1, beta):
    return 0.5*b**4*(diff(g3sCurlSym(rij, beta, kappa1, kappa2, Lambda), rij[eta2]) + diff(g3sCurlSym(rij, beta, kappa1, kappa2, eta2), rij[Lambda]))


def deldelg13sSym(rij,alpha,gamma1,gamma2,kappa2,kappa1,beta):
    return diff(g13sSym(rij,alpha,kappa2,kappa1,beta), rij[gamma1], rij[gamma2])

def lapg13sSym(rij,alpha,kappa2,kappa1,beta):
    return Laplacian(g13sSym(rij,alpha,kappa2,kappa1,beta))

def g3s3sSym(rij,alpha,gamma1,gamma2,kappa2,kappa1,beta):
    return 1/3*(deldelg13sSym(rij,alpha,gamma1,gamma2,kappa2,kappa1,beta) + deldelg13sSym(rij,gamma1,gamma2,alpha,kappa2,kappa1,beta) + 
                deldelg13sSym(rij,gamma2,alpha,gamma1,kappa2,kappa1,beta)) - 1/15*(
        KroneckerDelta[gamma1,gamma2]*lapg13sSym(rij, alpha, kappa2, kappa1, beta) + 
        KroneckerDelta[gamma2,alpha]*lapg13sSym(rij, gamma1, kappa2, kappa1, beta) + 
        KroneckerDelta[alpha,gamma1]*lapg13sSym(rij, gamma2, kappa2, kappa1, beta))

def G3s3s(rij,alpha,gamma1,gamma2,kappa2,kappa1,beta):
    return b**4*(g3s3sSym(rij,alpha,gamma1,gamma2,kappa2,kappa1,beta) + b**2/7*Laplacian(g3s3sSym(rij,alpha,gamma1,gamma2,kappa2,kappa1,beta)))


def G3a2s(rij, Lambda, kappa2, kappa1, beta):
    return -0.25*b**3*(diff(Curl(G,Lambda,beta), rij[kappa1], rij[kappa2]) + diff(Curl(G,Lambda,kappa1), rij[beta], rij[kappa2])
                       + diff(Curl(G,kappa2,beta), rij[kappa1], rij[Lambda]) + diff(Curl(G,kappa2,kappa1), rij[beta], rij[Lambda]))


def g3s2sSym(rij, alpha,gamma1,gamma2, kappa1, beta):
    return diff(g3s1Sym(rij, alpha,gamma1,gamma2,beta), rij[kappa1]) + diff(g3s1Sym(rij,alpha,gamma1,gamma2,kappa1), rij[beta])

def G3s2s(rij,alpha,gamma1,gamma2,kappa1,beta):
    return -0.5*b**3*(g3s2sSym(rij, alpha,gamma1,gamma2, kappa1, beta) + 6*b**2/35*Laplacian(g3s2sSym(rij, alpha,gamma1,gamma2, kappa1, beta)))


def G2a3a(rij, Lambda, mu, nu2): # sign change -> eps(mu,nu1,beta)
    return 2*b**3/3*diff(CurlCurl(G, Lambda, mu), rij[nu2])


def G2a3s(rij,Lambda,kappa2,kappa1,beta):
    return b**3*g3sCurlSym(rij, beta, kappa1, kappa2, Lambda)

def K2a2s(rij, Lambda, beta, gamma):
    return -4*pi*eta*b**3/3*(diff(Curl(G,Lambda,gamma), rij[beta]) + diff(Curl(G,Lambda,beta), rij[gamma])) 

### Matrix assembly

In [8]:
matG1s1s = Matrix.zeros(3,3)
for i in range(3):
    for j in range(3):
        matG1s1s[i,j] = simplify(G1s1s(rij, i, j))
        
print('G1s1s done')
        
matG2a1s = Matrix.zeros(3,3)
for i in range(3):
    for j in range(3):
        matG2a1s[i,j] = simplify(G2a1s(rij, i, j))
        
print('G2a1s done')
        
matG1s2a = Matrix.zeros(3,3)
for i in range(3):
    for j in range(3):
        matG1s2a[i,j] = simplify(G1s2a(rij, i, j))
        
print('G1s2a done')
        
matG2a2a = Matrix.zeros(3,3)
for i in range(3):
    for j in range(3):
        matG2a2a[i,j] = simplify(G2a2a(rij, i, j))
        
print('G2a2a done')
        
matG1s2s = Matrix.zeros(3,5)
j=0
for i in range(3):
    for k in range(2):
        for l in range(k,3):
            matG1s2s[i,j] = simplify(G1s2s(rij, i, k, l))
            j += 1
    j=0
    
print('G1s2s done')

matG2a2s = Matrix.zeros(3,5)
j=0
for i in range(3):
    for k in range(2):
        for l in range(k,3):
            matG2a2s[i,j] = simplify(G2a2s(rij, i, k, l))
            j += 1
    j=0
    
print('G2a2s done')
    
matG2s1s = Matrix.zeros(5,3)
i=0
for j in range(3):
    for k in range(2):
        for l in range(k,3):
            matG2s1s[i,j] = simplify(G2s1s(rij, k, l, j))
            i += 1
    i=0
    
print('G2s1s done')
    
matG2s2a = Matrix.zeros(5,3)
i=0
for j in range(3):
    for k in range(2):
        for l in range(k,3):
            matG2s2a[i,j] = simplify(G2s2a(rij, k, l, j))
            i += 1
    i=0
    
print('G2s2a done')
    
matG2s2s = Matrix.zeros(5,5)
i=0
j=0
for ki in range(2):
    for li in range(ki,3):
        for kj in range(2):
            for lj in range(kj,3):
                matG2s2s[i,j] = simplify(G2s2s(rij, ki, li, kj, lj))
                j += 1
        j=0
        i += 1 
        
print('G2s2s done')
        
matG2s3t = Matrix.zeros(5,3)
i=0
for j in range(3):
    for k in range(2):
        for l in range(k,3):
            matG2s3t[i,j] = simplify(G2s3t(rij, k, l, j))
            i += 1
    i=0
    
print('G2s3t done')
    
matG2s3a = Matrix.zeros(5,5)
i=0
j=0
for ki in range(2):
    for li in range(ki,3):
        for kj in range(2):
            for lj in range(kj,3):
                matG2s3a[i,j] = simplify(G2s3a(rij, ki, li, kj, lj))
                j += 1
        j=0
        i += 1 
        
print('G2s3a done')
        
matG3a2s = Matrix.zeros(5,5)
i=0
j=0
for ki in range(2):
    for li in range(ki,3):
        for kj in range(2):
            for lj in range(kj,3):
                matG3a2s[i,j] = simplify(G3a2s(rij, ki, li, kj, lj))
                j += 1
        j=0
        i += 1 
        
print('G3a2s done')
        
matG3a3a = Matrix.zeros(5,5)
i=0
j=0
for ki in range(2):
    for li in range(ki,3):
        for kj in range(2):
            for lj in range(kj,3):
                matG3a3a[i,j] = simplify(G3a3a(rij, ki, li, kj, lj))
                j += 1
        j=0
        i += 1 
        
print('G3a3a done')
        
matK1s2s = Matrix.zeros(3,5)
j=0
for i in range(3):
    for k in range(2):
        for l in range(k,3):
            matK1s2s[i,j] = simplify(K1s2s(rij, i, k, l))
            j += 1
    j=0
    
print('K1s2s done')
    
matK1s3t = Matrix.zeros(3,3)
for i in range(3):
    for j in range(3):
        matK1s3t[i,j] = simplify(K1s3t(rij, i, j))
        
print('K1s3t done')
        
matK2a2s = Matrix.zeros(3,5)
j=0
for i in range(3):
    for k in range(2):
        for l in range(k,3):
            matK2a2s[i,j] = simplify(K2a2s(rij, i, k, l))
            j += 1
    j=0  
    
print('K2a2s done')
    
matK2s2s = Matrix.zeros(5,5)
i=0
j=0
for ki in range(2):
    for li in range(ki,3):
        for kj in range(2):
            for lj in range(kj,3):
                matK2s2s[i,j] = simplify(K2s2s(rij, ki, li, kj, lj))
                j += 1
        j=0
        i += 1 
        
print('K2s2s done')
        
matK2s3t = Matrix.zeros(5,3)
i=0
for j in range(3):
    for k in range(2):
        for l in range(k,3):
            matK2s3t[i,j] = simplify(K2s3t(rij, k, l, j))
            i += 1
    i=0
    
print('K2s3t done')
    
matK3a2s = Matrix.zeros(5,5)
i=0
j=0
for ki in range(2):
    for li in range(ki,3):
        for kj in range(2):
            for lj in range(kj,3):
                matK3a2s[i,j] = simplify(K3a2s(rij, ki, li, kj, lj))
                j += 1
        j=0
        i += 1 
        
print('K3a2s done')
        
matG1s3t = Matrix.zeros(3,3)
for i in range(3):
    for j in range(3):
        matG1s3t[i,j] = simplify(G1s3t(rij, i, j))
        
print('G1s3t done')
        
matG1s3a = Matrix.zeros(3,5)
j=0
for i in range(3):
    for k in range(2):
        for l in range(k,3):
            matG1s3a[i,j] = simplify(G1s3a(rij, i, k, l))
            j += 1
    j=0  
    
print('G1s3a done')
    
matG2a3a = Matrix.zeros(3,5)
j=0
for i in range(3):
    for k in range(2):
        for l in range(k,3):
            matG2a3a[i,j] = simplify(G2a3a(rij, i, k, l))
            j += 1
    j=0  
    
print('G2a3a done')
    
matG3a1s = Matrix.zeros(5,3)
i=0
for j in range(3):
    for k in range(2):
        for l in range(k,3):
            matG3a1s[i,j] = simplify(G3a1s(rij, k, l, j))
            i += 1
    i=0
    
print('G3a1s done')
    
matG3a2a = Matrix.zeros(5,3)
i=0
for j in range(3):
    for k in range(2):
        for l in range(k,3):
            matG3a2a[i,j] = simplify(G3a2a(rij, k, l, j))
            i += 1
    i=0
    
print('G3a2a done')
    
matG1s3s = Matrix.zeros(3,7)
j=0
for i in range(3):
    for k in range(2):
        for l in range(k,2):
            for m in range(l,3):
                matG1s3s[i,j] = simplify(G1s3s(rij,i, k, l, m))
                j += 1
    j=0
    
print('G1s3s done')
    
matG2a3s = Matrix.zeros(3,7)
j=0
for i in range(3):
    for k in range(2):
        for l in range(k,2):
            for m in range(l,3):
                matG2a3s[i,j] = simplify(G2a3s(rij,i, k, l, m))
                j += 1
    j=0
    
print('G2a3s done')
    
matG3s1s = Matrix.zeros(7,3)
i=0
for j in range(3):
    for k in range(2):
        for l in range(k,2):
            for m in range(l,3):
                matG3s1s[i,j] = simplify(G3s1s(rij, k, l, m, j))
                i += 1
    i=0
    
print('G3s1s done')
    
matG3s2a = Matrix.zeros(7,3)
i=0
for j in range(3):
    for k in range(2):
        for l in range(k,2):
            for m in range(l,3):
                matG3s2a[i,j] = simplify(G3s2a(rij, k, l, m, j))
                i += 1
    i=0
    
print('G3s2a done')
    
matG2s3s = Matrix.zeros(5,7)
i=0
j=0
for ki in range(2):
    for li in range(ki,3):
        for kj in range(2):
            for lj in range(kj,2):
                for mj in range(lj,3):
                    matG2s3s[i,j] = simplify(G2s3s(rij, ki, li, kj, lj, mj))
                    j += 1
        j=0
        i += 1
        
print('G2s3s done')
        
matG3a3s = Matrix.zeros(5,7)
i=0
j=0
for ki in range(2):
    for li in range(ki,3):
        for kj in range(2):
            for lj in range(kj,2):
                for mj in range(lj,3):
                    matG3a3s[i,j] = simplify(G3a3s(rij, ki, li, kj, lj, mj))
                    j += 1
        j=0
        i += 1
        
print('G3a3s done')
        
matG3s2s = Matrix.zeros(7,5)
i=0
j=0
for ki in range(2):
    for li in range(ki,2):
        for mi in range(li,3):
            for kj in range(2):
                for lj in range(kj,3):
                    matG3s2s[i,j] = simplify(G3s2s(rij, ki, li, mi, kj, lj))
                    j += 1
            j=0
            i += 1
            
print('G3s2s done')
            
matG3s3a = Matrix.zeros(7,5)
i=0
j=0
for ki in range(2):
    for li in range(ki,2):
        for mi in range(li,3):
            for kj in range(2):
                for lj in range(kj,3):
                    matG3s3a[i,j] = simplify(G3s3a(rij, ki, li, mi, kj, lj))
                    j += 1
            j=0
            i += 1
            
print('G3s3t done')
            
matG3s3t = Matrix.zeros(7,3)
i=0
for j in range(3):
    for k in range(2):
        for l in range(k,2):
            for m in range(l,3):
                matG3s3t[i,j] = simplify(G3s3t(rij, k, l, m, j))
                i += 1
    i=0
    
print('G3s3t done')
    
matG3s3s = Matrix.zeros(7,7)
i=0
j=0
for ki in range(2):
    for li in range(ki,2):
        for mi in range(li,3):
            for kj in range(2):
                for lj in range(kj,2):
                    for mj in range(lj,3):
                        matG3s3s[i,j] = simplify(G3s3s(rij, ki, li, mi, kj, lj, mj))
                        j += 1
            j=0
            i += 1
            
print('G3s3s done')

matK3s2s = Matrix.zeros(7,5)
i=0
j=0
for ki in range(2):
    for li in range(ki,2):
        for mi in range(li,3):
            for kj in range(2):
                for lj in range(kj,3):
                    matK3s2s[i,j] = simplify(K3s2s(rij, ki, li, mi, kj, lj))
                    j += 1
            j=0
            i += 1
            
print('K3s2s done')
            
matK3s3t = Matrix.zeros(7,3)
i=0
for j in range(3):
    for k in range(2):
        for l in range(k,2):
            for m in range(l,3):
                matK3s3t[i,j] = simplify(K3s3t(rij, k, l, m, j))
                i += 1
    i=0
    
print('K3s3t done')
print('...done')

G1s1s done
G2a1s done
G1s2a done
G2a2a done
G1s2s done
G2a2s done
G2s1s done
G2s2a done
G2s2s done
G2s3t done
G2s3a done
G3a2s done
G3a3a done
K1s2s done
K1s3t done
K2a2s done
K2s2s done
K2s3t done
K3a2s done
G1s3t done
G1s3a done
G2a3a done
G3a1s done
G3a2a done
G1s3s done
G2a3s done
G3s1s done
G3s2a done
G2s3s done
G3a3s done
G3s2s done
G3s3t done
G3s3t done
G3s3s done
K3s2s done
K3s3t done
...done


Create matrices that are zero

In [28]:
matG3t1s = Matrix(ZeroMatrix(3,3))
matG3t2a = Matrix(ZeroMatrix(3,3))

matG3t2s = Matrix(ZeroMatrix(3,5))
matG3t3t = Matrix(ZeroMatrix(3,3))

matG3t3a = Matrix(ZeroMatrix(3,5))
matG3t3s = Matrix(ZeroMatrix(3,7))

matK3t2s = Matrix(ZeroMatrix(3,5))
matK3t3t = Matrix(ZeroMatrix(3,3))

matK3t3a = Matrix(ZeroMatrix(3,5))
matK3t3s = Matrix(ZeroMatrix(3,7))

matG2a3t = Matrix(ZeroMatrix(3,3))
matG3a3t = Matrix(ZeroMatrix(5,3))

matK2a3t = Matrix(ZeroMatrix(3,3))
matK3a3t = Matrix(ZeroMatrix(5,3))

Need the following objects

GH1s(force), GH2a(torque), KHH(VH), GHH(FH), 

G1s1s(force), G2a1s(force), G1s2a(torque), G2a2a(torque)
G1sH(FH), K1sH(VH), G2aH(FH), K2aH(FH)

Want to assemble them locally and then print them to a py file. keep placeholders such as force (array of len 3), FH (array of len 20) etc. create these first...

In [48]:
## In python code will be force[i,0], so have to create force as np.zeros([3*Np,1]) object instead of np.zeros([3*Np])
force  = Matrix(MatrixSymbol('force',3,1)) ##create an actual Matrix from it, have to pass to 'Matrix' 
torque = Matrix(MatrixSymbol('torque',3,1))
VH     = Matrix(MatrixSymbol('VH',20,1))
FH     = Matrix(MatrixSymbol('FH',20,1))

Create block matrices, then multiply with vectors and then print as functions

In [65]:
matG1sH = Matrix(BlockMatrix([matG1s2s, matG1s3t, matG1s3a, matG1s3s]))
matG2aH = Matrix(BlockMatrix([matG2a2s, matG2a3t, matG2a3a, matG2a3s]))

matK1sH = Matrix(BlockMatrix([matK1s2s, matK1s3t, Matrix(ZeroMatrix(3,12))]))
matK2aH = Matrix(BlockMatrix([matK2a2s, matK2a3t, Matrix(ZeroMatrix(3,12))]))

matGH1s = Matrix(BlockMatrix([[matG2s1s],
                              [matG3t1s],
                              [matG3a1s],
                              [matG3s1s]]))
matGH2a = Matrix(BlockMatrix([[matG2s2a],
                              [matG3t2a],
                              [matG3a2a],
                              [matG3s2a]]))

matGHH = Matrix(BlockMatrix([[matG2s2s, matG2s3t, matG2s3a, matG2s3s],
                             [matG3t2s, matG3t3t, matG3t3a, matG3t3s],
                             [matG3a2s, matG3a3t, matG3a3a, matG3a3s],
                             [matG3s2s, matG3s3t, matG3s3a, matG3s3s]]))

matKHH_nonzero = Matrix(BlockMatrix([[matK2s2s, matK2s3t],
                                     [matK3t2s, matK3t3t],
                                     [matK3a2s, matK3a3t],
                                     [matK3s2s, matK3s3t]]))
matKHH = Matrix(BlockMatrix([matKHH_nonzero, Matrix(ZeroMatrix(20,12))]))

##diagonal elements
g1s = 1/(6*pi*eta*b)
g2a = 1/(4*pi*eta*b)

g2s = 3/(20*pi*eta*b)
g3t = 1/(2*pi*eta*b)
g3a = 3/(2*pi*eta*b)
g3s = 6/(7*pi*eta*b)

matGoHH = diag(g2s,g2s,g2s,g2s,g2s, g3t,g3t,g3t, g3a,g3a,g3a,g3a,g3a, g3s,g3s,g3s,g3s,g3s,g3s,g3s)

halfMinusk2s = 0.6
halfMinusk3t = 0.4
halfMinusk3a = 0.8
halfMinusk3s = 19/35

matKoHH = diag(halfMinusk2s,halfMinusk2s,halfMinusk2s,halfMinusk2s,halfMinusk2s, halfMinusk3t,halfMinusk3t,halfMinusk3t,
               halfMinusk3a,halfMinusk3a,halfMinusk3a,halfMinusk3a,halfMinusk3a, halfMinusk3s,halfMinusk3s,halfMinusk3s,halfMinusk3s,halfMinusk3s,halfMinusk3s,halfMinusk3s)

Create vectors instead of constructing the full matrix

In [70]:
vecGH1sF = matGH1s*force
vecGH2aT = matGH2a*torque

vecKHHVH_diag = matKoHH*VH
vecKHHVH = matKHH*VH

vecGHHFH_diag = matGoHH*FH
vecGHHFH = matGHH*FH

vecG1s1sF = matG1s1s*force
vecG2a1sF = matG2a1s*force

vecG1s2aT = matG1s2a*torque
vecG2a2aT = matG2a2a*torque

vecG1sHFH = matG1sH*FH
vecG2aHFH = matG2aH*FH

vecK1sHVH = matK1sH*VH
vecK2aHVH = matK2aH*VH

_____________

### Write symbolic results to file as reusable python functions

In [294]:
ME_file = "test.py" #"freeSpaceME.py"

with open(ME_file, "w") as text_file:
    print("import numpy", file=text_file)
    print("PI = 3.14159265359\n", file=text_file)
    
    print("def G1s1sF(xij,yij,zij, b,eta, force):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecG1s1sF),(3,)).tolist()), file=text_file)
    print("def G2a1sF(xij,yij,zij, b,eta, force):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecG2a1sF),(3,)).tolist()), file=text_file)
    
    print("def G1s2aT(xij,yij,zij, b,eta, torque):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecG1s2aT),(3,)).tolist()), file=text_file)
    print("def G2a2aT(xij,yij,zij, b,eta, torque):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecG2a2aT),(3,)).tolist()), file=text_file)
    
    print("def GH1sF(xij,yij,zij, b,eta, force):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecGH1sF),(20,)).tolist()), file=text_file)
    print("def GH2aT(xij,yij,zij, b,eta, torque):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecGH2aT),(20,)).tolist()), file=text_file)
    
    print("def KoHHVH(b,eta, VH):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecKHHVH_diag),(20,)).tolist()), file=text_file)
    print("def KHHVH(xij,yij,zij, b,eta, VH):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecKHHVH),(20,)).tolist()), file=text_file)
    
    print("def GoHHFH(b,eta, FH):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecGHHFH_diag),(20,)).tolist()), file=text_file)
    print("def GHHFH(xij,yij,zij, b,eta, FH):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecGHHFH),(20,)).tolist()), file=text_file)
    
    print("def G1sHFH(xij,yij,zij, b,eta, FH):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecG1sHFH),(3,)).tolist()), file=text_file)
    print("def G2aHFH(xij,yij,zij, b,eta, FH):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecG2aHFH),(3,)).tolist()), file=text_file)
    
    print("def K1sHVH(xij,yij,zij, b,eta, VH):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecK1sHVH),(3,)).tolist()), file=text_file)
    print("def K2aHVH(xij,yij,zij, b,eta, VH):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecK2aHVH),(3,)).tolist()), file=text_file)
    
    
    
## replace things for easier usage with python     
with open(ME_file, 'r') as file:
    filedata = file.read()
    
filedata = filedata.replace('pi', 'PI')
filedata = filedata.replace('sqrt', 'numpy.sqrt')

filedata = re.sub("force\[(\d), 0\]", r"force[\1]", filedata)
filedata = re.sub("torque\[(\d), 0\]", r"torque[\1]", filedata)
filedata = re.sub("VH\[(\d||\d\d), 0\]", r"VH[\1]", filedata)
filedata = re.sub("FH\[(\d||\d\d), 0\]", r"FH[\1]", filedata)

with open(ME_file, 'w') as file:
    file.write(filedata)

In [288]:
importlib.reload(test)

<module 'test' from '/store/SOFT/gt369/PhD/pystokes/examples/test-gt/test.py'>

In [289]:
fff = np.array([0, 0, 1])
test.G1s1sF(1,1,1, 1,1, fff)

array([0.00255245, 0.00255245, 0.03062938])

In [12]:
## try to assemble one of the matrices

In [13]:
matG1s1s = Matrix.zeros(3,3)
for i in range(3):
    for j in range(3):
        matG1s1s[i,j] = simplify(G1s1s(rij, i, j))

In [14]:
matG1s1s ##works and as far as I can see, matches the Mathematica result

Matrix([
[(-2*b**2*(2*xij**2 - yij**2 - zij**2) + 3*(xij**2 + yij**2 + zij**2)*(2*xij**2 + yij**2 + zij**2))/(24*pi*eta*(xij**2 + yij**2 + zij**2)**(5/2)),                                                         xij*yij*(-2*b**2 + xij**2 + yij**2 + zij**2)/(8*pi*eta*(xij**2 + yij**2 + zij**2)**(5/2)),                                                         xij*zij*(-2*b**2 + xij**2 + yij**2 + zij**2)/(8*pi*eta*(xij**2 + yij**2 + zij**2)**(5/2))],
[                                                       xij*yij*(-2*b**2 + xij**2 + yij**2 + zij**2)/(8*pi*eta*(xij**2 + yij**2 + zij**2)**(5/2)), (-2*b**2*(-xij**2 + 2*yij**2 - zij**2) + 3*(xij**2 + yij**2 + zij**2)*(xij**2 + 2*yij**2 + zij**2))/(24*pi*eta*(xij**2 + yij**2 + zij**2)**(5/2)),                                                         yij*zij*(-2*b**2 + xij**2 + yij**2 + zij**2)/(8*pi*eta*(xij**2 + yij**2 + zij**2)**(5/2))],
[                                                       xij*zij*(-2*b**2 + xij**2 + yij**2 + zij**2)/(8*pi*

In [15]:
## when printing to python code, might have to be careful with greek characters that sympy is printing so nicely
pycode(matG1s1s)

'ImmutableDenseMatrix([[(1/24)*(-2*b**2*(2*xij**2 - yij**2 - zij**2) + 3*(xij**2 + yij**2 + zij**2)*(2*xij**2 + yij**2 + zij**2))/(math.pi*eta*(xij**2 + yij**2 + zij**2)**(5/2)), (1/8)*xij*yij*(-2*b**2 + xij**2 + yij**2 + zij**2)/(math.pi*eta*(xij**2 + yij**2 + zij**2)**(5/2)), (1/8)*xij*zij*(-2*b**2 + xij**2 + yij**2 + zij**2)/(math.pi*eta*(xij**2 + yij**2 + zij**2)**(5/2))], [(1/8)*xij*yij*(-2*b**2 + xij**2 + yij**2 + zij**2)/(math.pi*eta*(xij**2 + yij**2 + zij**2)**(5/2)), (1/24)*(-2*b**2*(-xij**2 + 2*yij**2 - zij**2) + 3*(xij**2 + yij**2 + zij**2)*(xij**2 + 2*yij**2 + zij**2))/(math.pi*eta*(xij**2 + yij**2 + zij**2)**(5/2)), (1/8)*yij*zij*(-2*b**2 + xij**2 + yij**2 + zij**2)/(math.pi*eta*(xij**2 + yij**2 + zij**2)**(5/2))], [(1/8)*xij*zij*(-2*b**2 + xij**2 + yij**2 + zij**2)/(math.pi*eta*(xij**2 + yij**2 + zij**2)**(5/2)), (1/8)*yij*zij*(-2*b**2 + xij**2 + yij**2 + zij**2)/(math.pi*eta*(xij**2 + yij**2 + zij**2)**(5/2)), (1/24)*(-2*b**2*(-xij**2 - yij**2 + 2*zij**2) + 3*(xij**2 + y

In [16]:
from sympy.printing.numpy import NumPyPrinter

In [17]:
NumPyPrinter().doprint(matG1s1s)

'numpy.array([[(1/24)*(-2*b**2*(2*xij**2 - yij**2 - zij**2) + 3*(xij**2 + yij**2 + zij**2)*(2*xij**2 + yij**2 + zij**2))/(numpy.pi*eta*(xij**2 + yij**2 + zij**2)**(5/2)), (1/8)*xij*yij*(-2*b**2 + xij**2 + yij**2 + zij**2)/(numpy.pi*eta*(xij**2 + yij**2 + zij**2)**(5/2)), (1/8)*xij*zij*(-2*b**2 + xij**2 + yij**2 + zij**2)/(numpy.pi*eta*(xij**2 + yij**2 + zij**2)**(5/2))], [(1/8)*xij*yij*(-2*b**2 + xij**2 + yij**2 + zij**2)/(numpy.pi*eta*(xij**2 + yij**2 + zij**2)**(5/2)), (1/24)*(-2*b**2*(-xij**2 + 2*yij**2 - zij**2) + 3*(xij**2 + yij**2 + zij**2)*(xij**2 + 2*yij**2 + zij**2))/(numpy.pi*eta*(xij**2 + yij**2 + zij**2)**(5/2)), (1/8)*yij*zij*(-2*b**2 + xij**2 + yij**2 + zij**2)/(numpy.pi*eta*(xij**2 + yij**2 + zij**2)**(5/2))], [(1/8)*xij*zij*(-2*b**2 + xij**2 + yij**2 + zij**2)/(numpy.pi*eta*(xij**2 + yij**2 + zij**2)**(5/2)), (1/8)*yij*zij*(-2*b**2 + xij**2 + yij**2 + zij**2)/(numpy.pi*eta*(xij**2 + yij**2 + zij**2)**(5/2)), (1/24)*(-2*b**2*(-xij**2 - yij**2 + 2*zij**2) + 3*(xij**2 + yi